In [ ]:
import json
import numpy as np
import pandas as pd
import folium

# データ読み込み
df = pd.read_json("https://covid.ourworldindata.org/data/owid-covid-data.json")
df = df[df.columns.drop(list(df.filter(regex='OWID')))] 

In [ ]:
# 難：日付データ"data"のうち，各国の最新データを元にデータフレームを作成

countries = df.columns
date_keys = pd.DataFrame(df['USA']['data']).columns

df2 = pd.DataFrame(columns = countries, index = date_keys)
for column in countries:
    for indexs in date_keys:
        try:
            df2[column][indexs] = df[column]['data'][-7][indexs]
        except LookupError:
            df2[column][indexs] = np.nan

# 二つのデータフレームをマージ
df_all = pd.concat([df,df2]).T
df_all['index'] = df_all.index # foliumのために国名コードのindexもcolumnに追加する

# df_allの表示
df_all

In [ ]:
df_all.columns # columnの表示

In [ ]:
# 国形状のデータを取得する
url = "https://raw.githubusercontent.com/python-visualization/folium/master/examples/data"
world_geo = f"{url}/world-countries.json"

# foliumの表示の初期設定
m = folium.Map(location=[0, 0], zoom_start=2)

# foliumのオプション
folium.Choropleth(
    geo_data=world_geo, # 世界の国の形状を用いる
    name="choropleth",
    data=df_all, # OWIDのデータを使う
    columns=["index", "total_cases_per_million"], # dfのうちindex（国名コード），total_cases_per_millionのcolumnを使う
    key_on="feature.id", # world_geoのfeature.idと国名コードが一致するとき，その国を色塗り
    fill_color="YlOrRd", # 色の設定
    fill_opacity=0.7,
    line_opacity=0.2,
    nan_fill_color='black', # データがない国の色を設定
    legend_name="total_cases_per_million", # 凡例のラベル
).add_to(m)

folium.LayerControl().add_to(m)

m # 表示